In [1]:
import pandas as pd
import numpy as np
#articles.csv available  at https://www.kaggle.com/marlesson/news-of-the-site-folhauol
articles = pd.read_csv('articles.csv')
articles.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [2]:
articles.category.value_counts(normalize=True)

poder                           0.131826
colunas                         0.129432
mercado                         0.125529
esporte                         0.118106
mundo                           0.102542
cotidiano                       0.101567
ilustrada                       0.097843
opiniao                         0.027087
paineldoleitor                  0.024010
saopaulo                        0.023675
tec                             0.013529
tv                              0.012822
educacao                        0.012679
turismo                         0.011392
ilustrissima                    0.008446
ciencia                         0.007991
equilibrioesaude                0.007854
sobretudo                       0.006327
bbc                             0.005866
folhinha                        0.005244
empreendedorsocial              0.005034
comida                          0.004957
asmais                          0.003280
ambiente                        0.002939
seminariosfolha 

In [5]:
articles.category.value_counts()

poder                           22022
colunas                         21622
mercado                         20970
esporte                         19730
mundo                           17130
cotidiano                       16967
ilustrada                       16345
opiniao                          4525
paineldoleitor                   4011
saopaulo                         3955
tec                              2260
tv                               2142
educacao                         2118
turismo                          1903
ilustrissima                     1411
ciencia                          1335
equilibrioesaude                 1312
sobretudo                        1057
bbc                               980
folhinha                          876
empreendedorsocial                841
comida                            828
asmais                            548
ambiente                          491
seminariosfolha                   379
serafina                          334
o-melhor-de-

In [36]:
articles.loc[articles.category=='colunas'].title.iloc[:10].values

array(['50 anos depois da morte, o Che é apenas uma foto na camiseta',
       'Afinal, nosso voto tem preço?',
       'Cientista dá psicodélico para minicérebros, e eles gostam',
       'Reestruturação da Sabesp deverá ser levada à Justiça por setor de limpeza',
       'Preservar passado é um dos grandes desafios dos processos de revitalização',
       'Não à Copa sem Lionel Messi',
       'UNE processa Lollapalooza por suspeita de burlar lei da meia entrada',
       "Esquerda pode tomar democratas, que vivem 'cabo-de-guerra', diz 'NYT'",
       'Putin encontra no Oriente Médio maior triunfo diplomático',
       'O candidato da situação'], dtype=object)

Let's keep only the 10'st commom headline categories.

In [3]:
categories = articles.category.value_counts().index[:10]
filtered_articles = articles.loc[articles.category.isin(categories)]
filtered_articles.shape

(147277, 6)

In [44]:
filtered_articles.category.value_counts(normalize=True)

poder             0.149528
colunas           0.146812
mercado           0.142385
esporte           0.133965
mundo             0.116311
cotidiano         0.115205
ilustrada         0.110981
opiniao           0.030724
paineldoleitor    0.027234
saopaulo          0.026854
Name: category, dtype: float64

In [52]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
zeros = np.zeros(10)
zeros[np.where(categories=='colunas')] = 1
zeros

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
import sentencepiece as spm

In [16]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [34]:
X = filtered_articles.title.apply(lambda x: sp.encode_as_ids(x))

In [50]:
def build_target(category):
    zeros = np.zeros(10)
    zeros[np.where(categories==category)] = 1
    return zeros

In [59]:
np.where(categories=='poder')[0][0]

0

In [60]:
#y = filtered_articles.category.apply(lambda y: build_target(y))
y = filtered_articles.category.apply(lambda y: np.where(categories==y)[0][0])

In [61]:
y[:10]

0     0
1     6
2     0
3     6
4     2
5     4
6     0
7     3
8     0
10    0
Name: category, dtype: int64

In [41]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [66]:
import tensorflow as tf

In [78]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [80]:
y_true = [[0., 1.], [0., 0.]] 
y_pred = [[0., 1.], [0., 0.]] 
bce(y_true, y_pred).numpy()

0.6931472